Date: June 5, 2025  
Author: Diane Menuz  
Looked at daily and monthly daytime energy closure for all sites with data available between March and October 2024


Most recent data for any site in the database is from March 2025

Sites with data between March and October 2024: US-CdM, US-UTD, US-UTE, US-UTJ, US-UTM, US-UTV, US-UTW

# Parameters and Libraries

## Parameters

In [8]:
table = 'amfluxeddy'

micromet_path= "C:/Users/dmenuz/Documents/scripts/MicroMet/src/micromet"
loggerloader_path= "C:/Users/dmenuz/Documents/scripts/loggerloader"

## Libraries

In [9]:
import pandas as pd
import numpy as np
import importlib.util
import plotly.graph_objects as go
import plotly.express as px


from sqlalchemy import create_engine, inspect
from urllib.parse import quote  
import requests
import json

In [10]:
# import functions
import sys
sys.path.append(micromet_path)
import graphs as mm_graphs

sys.path.append(loggerloader_path)
import loggerloader as ll

## Database Connection

In [ ]:
spec = importlib.util.spec_from_file_location("dbconnect", "C:/Users/dmenuz/Documents/scripts/supporting_files/dbconnect_cloud.py")
dbconnect = importlib.util.module_from_spec(spec)
spec.loader.exec_module(dbconnect)

engine = dbconnect.postconn_ugs_read()

# Daily and Monthly Data

In [ ]:
start_date = '2024-03-01'
end_date = '2024-10-31'
monthly_dict = {}
growing_season_dict = {}


In [ ]:
# pull data from database to get list of stations with data
query = "SELECT DiSTINCT stationid FROM groundwater.amfluxeddy WHERE datetime_start >= %(start_date)s and datetime_start <= %(end_date)s and netrad >-9999 and g>-9999"
stations = pd.read_sql(query, engine, params={'start_date': start_date, 'end_date': end_date})

In [49]:
for station in stations.stationid:
    print(station)
    query = "SELECT * FROM groundwater.amfluxeddy WHERE stationid = %(sid)s and datetime_start >= %(start_date)s and datetime_start <= %(end_date)s"
    params={'sid': station, 'start_date': start_date, 'end_date': end_date}

    # set index, drop na values
    eddy = pd.read_sql(query, engine, params=params)
    eddy.set_index('datetime_start', inplace=True)
    eddy.replace(-9999,np.nan,inplace=True)
    eddy_dropna = eddy.dropna(subset = ['netrad', 'h', 'le', 'g'], how='any')
    eddy.head()

    # select hours that are in good days (48 records)
    drop_threshold=48
    daily_dat = eddy_dropna.resample('D').count()
    daily_net_enough_records = daily_dat[daily_dat.netrad>=drop_threshold]
    hourly_gooddays = eddy_dropna[eddy_dropna.index.floor('D').isin(daily_net_enough_records.index)]

    # summarize to monthly
    monthly_clean = hourly_gooddays.resample('MS').agg({
        'le': 'sum',
        'h': 'sum',
        'netrad': 'sum',
        'g': 'sum',
        'stationid':'count'
    })

    monthly_clean['closure'] = (monthly_clean['le']+monthly_clean['h'])/(monthly_clean['netrad']-monthly_clean['g'])
    monthly_clean['days_in_month'] = monthly_clean['stationid']/48
    monthly_dict[station] = monthly_clean[['closure', 'days_in_month']]

    # summarize to growing season (or annual)
    growing_season = hourly_gooddays.resample('YE').agg({
        'le': 'sum',
        'h': 'sum',
        'netrad': 'sum',
        'g': 'sum',
        'stationid':'count',
    })


    growing_season['closure'] = (growing_season['le']+growing_season['h'])/(growing_season['netrad']-growing_season['g'])
    growing_season_dict[station] = growing_season['closure'].iloc[0]


    

US-UTD
US-UTV
US-CdM
US-UTE
US-UTJ
US-UTM
US-UTW


In [68]:
monthly_dat = pd.concat(monthly_dict)
monthly_dat.reset_index(inplace=True)

pivoted_df = monthly_dat.pivot_table(
    index='datetime_start',
    columns='level_0',
    values=['days_in_month', 'closure'],
    aggfunc='first'
)
pivoted_df.to_csv(r'C:\Users\dmenuz\Desktop\temp\closure.csv')

In [71]:
growing_season_dict


{'US-UTD': np.float64(0.6397409110828715),
 'US-UTV': np.float64(0.736573045050875),
 'US-CdM': np.float64(1.0633013953635682),
 'US-UTE': np.float64(0.8272994929715909),
 'US-UTJ': np.float64(0.5777436614735132),
 'US-UTM': np.float64(1.0090151859946075),
 'US-UTW': np.float64(0.6217252583844296)}